In [ ]:
#  Core functions (SBERT + cosine + calibration)
from typing import Dict
import json, os
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Global model (singleton)
_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
_model_singleton = None

def get_model() -> SentenceTransformer:
    global _model_singleton
    if _model_singleton is None:
        _model_singleton = SentenceTransformer(_MODEL_NAME)
    return _model_singleton

# Calibration storage
CALIB_PATH = "calibration.json"
_DEF_MIN, _DEF_MAX = -1.0, 1.0

def _cosine(a: np.ndarray, b: np.ndarray) -> float:
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

def _scale_to_unit_interval(value: float, vmin: float, vmax: float) -> float:
    if vmax <= vmin:
        return (value + 1.0) / 2.0
    x = (value - vmin) / (vmax - vmin)
    return float(max(0.0, min(1.0, x)))

def _load_calibration() -> Dict[str, float]:
    if os.path.exists(CALIB_PATH):
        with open(CALIB_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"cos_min": _DEF_MIN, "cos_max": _DEF_MAX}

# global calib cache
_CALIB = _load_calibration()

def compute_similarity(text1: str, text2: str) -> float:
    """
    Returns a calibrated semantic similarity score in [0, 1].
    """
    model = get_model()
    emb = model.encode([text1, text2], normalize_embeddings=False)
    cos = _cosine(emb[0], emb[1])
    score = _scale_to_unit_interval(cos, _CALIB.get("cos_min", _DEF_MIN), _CALIB.get("cos_max", _DEF_MAX))
    # Light smoothing toward 0.5 for very short texts
    length_penalty = min(max((len(text1) + len(text2)) / 1000.0, 0.0), 1.0)
    score = float(0.1 * 0.5 * (1.0 - length_penalty) + score * (1.0 - 0.1 * (1.0 - length_penalty)))
    return round(score, 4)

print("Model + similarity function ready.")


Model + similarity function ready.


In [ ]:


# Option B: If file already present / in Drive, set path directly:
csv_path = "DataNeuron_Text_Similarity.csv"  # change if needed

def calibrate_from_csv(csv_path: str, out_json: str = "calibration.json"):
    df = pd.read_csv(csv_path)
    if not {"text1", "text2"}.issubset(df.columns):
        raise ValueError("CSV must contain columns: text1, text2")
    model = get_model()
    cos_vals = []
    batch = 64
    for i in range(0, len(df), batch):
        chunk = df.iloc[i:i+batch]
        embs1 = model.encode(chunk["text1"].tolist(), normalize_embeddings=False)
        embs2 = model.encode(chunk["text2"].tolist(), normalize_embeddings=False)
        for a, b in zip(embs1, embs2):
            cos_vals.append(_cosine(a, b))
    cos_min = float(np.percentile(cos_vals, 1))
    cos_max = float(np.percentile(cos_vals, 99))
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump({"cos_min": cos_min, "cos_max": cos_max}, f, indent=2)
    print(f"Wrote {out_json} with cos_min={cos_min:.4f}, cos_max={cos_max:.4f}")
    global _CALIB
    _CALIB = {"cos_min": cos_min, "cos_max": cos_max}

try:
    calibrate_from_csv(csv_path)
except FileNotFoundError:
    print("CSV not found. Upload it (see Option A) or fix the path, then re-run this cell.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Wrote calibration.json with cos_min=-0.1035, cos_max=0.4553


In [ ]:
#  Quick test
text1 = "nuclear body seeks new tech"   # change as needed
text2 = "terror suspects face arrest"   # change as needed
print("similarity score:", compute_similarity(text1, text2))


similarity score: 0.1176


In [ ]:
# Batch scoring: read CSV with (text1, text2) and write similarity column
in_csv = "DataNeuron_Text_Similarity.csv"
out_csv = "scored_pairs.csv"
try:
    df = pd.read_csv(in_csv)
    if not {"text1", "text2"}.issubset(df.columns):
        raise ValueError("CSV must contain columns: text1, text2")
    scores = []
    model = get_model()
    batch = 64
    for i in range(0, len(df), batch):
        chunk = df.iloc[i:i+batch]
        embs1 = model.encode(chunk["text1"].tolist(), normalize_embeddings=False)
        embs2 = model.encode(chunk["text2"].tolist(), normalize_embeddings=False)
        for a, b in zip(embs1, embs2):
            cos = _cosine(a, b)
            score = _scale_to_unit_interval(cos, _CALIB.get("cos_min", -1.0), _CALIB.get("cos_max", 1.0))
            # keep batch smoothing same rule
            length_penalty = 0.0
            score = float(0.1 * 0.5 * (1.0 - length_penalty) + score * (1.0 - 0.1 * (1.0 - length_penalty)))
            scores.append(round(score, 4))
    df["similarity"] = scores
    df.to_csv(out_csv, index=False)
    print(f"Wrote {out_csv} with {len(df)} rows")
except FileNotFoundError:
    print("Input CSV not found. Upload or set a correct path and re-run.")


Wrote scored_pairs.csv with 3000 rows


In [ ]:
pip install pyngrok

In [ ]:
# FastAPI server with public endpoint via ngrok
import nest_asyncio, uvicorn, threading
from fastapi import FastAPI
from pydantic import BaseModel, Field
from pyngrok import ngrok


NGROK_TOKEN = "2uoo6zGCOe7cYc2sYAtcrKSTUzO_3iT9iR82wD4N1TSY6GXE9"
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

nest_asyncio.apply()
api_app = FastAPI(title="DataNeuron STS API (Colab)")

class STSRequest(BaseModel):
    text1: str = Field(..., description="First input text")
    text2: str = Field(..., description="Second input text")

@api_app.get("/")
def root():
    return {"message": "DataNeuron STS API is running. POST /predict with 'text1' and 'text2'."}

@api_app.post("/predict")
def predict(req: STSRequest):
    try:
        score = compute_similarity(req.text1, req.text2)
        return {"similarity score": score}
    except Exception as e:
        return {"error": str(e)}

def run_server():
    uvicorn.run(api_app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

public_url = ngrok.connect(8000, bind_tls=True).public_url
print("Public API base:", public_url)
print("Test POST endpoint:", public_url + "/predict")
print("Example curl:")
print(f"curl -X POST {public_url}/predict -H 'Content-Type: application/json' -d '{{\"text1\":\"hello\",\"text2\":\"hi\"}}'")


INFO:     Started server process [278]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public API base: https://d600424a7cf8.ngrok-free.app
Test POST endpoint: https://d600424a7cf8.ngrok-free.app/predict
Example curl:
curl -X POST https://d600424a7cf8.ngrok-free.app/predict -H 'Content-Type: application/json' -d '{"text1":"hello","text2":"hi"}'


In [ ]:
# Gradio UI (shareable)
import gradio as gr

def gr_compute(t1, t2):
    if not t1 or not t2:
        return 0.0
    return compute_similarity(t1, t2)

with gr.Blocks() as demo:
    gr.Markdown("# DataNeuron STS — Demo")
    t1 = gr.Textbox(label="Text 1")
    t2 = gr.Textbox(label="Text 2")
    out = gr.Number(label="similarity score [0,1]")
    btn = gr.Button("Compute")
    btn.click(gr_compute, [t1, t2], out)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f7faaca88e20c4eed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
